In [ ]:
# Here the code asks the user for frame no. where furrow initiation starts and cytokinesis ends for each embryos(using the cortical plane video). 
# Then using these information, MSD for all the embryos are cailculated separately and then finally the lag time and MSD is merged in a dataframe for all embryos.

In [1]:
import pandas as pd
import numpy as np
from collections import defaultdict
import math as m

In [2]:
# The function 'compute_msd_true_average' takes the spot statistics data of tracked puncta,time interval between frames,first frame to include in tracked spots, last frame to include in tracked spots. 
# And then this function computes MSD
def compute_msd_true_average(data, t_gap ,frame_start, frame_end, max_lag=None):
    
    data_in_range = data[(data['Frame'] >= frame_start) & (data['Frame'] <= frame_end)]   #The data is filtered to restrict tracks to a time window(during furrow ingression)
    
    # The filtered data is then grouped by TrackID
    tracks = data_in_range.groupby('Track ID')

    # Dictionary to collect all displacements per lag
    displacements_by_lag = defaultdict(list)

    for track_id, group in tracks:    # This groups the filtered dataset by each unique Track ID (each tracked punctae) and iterates over each tracked particle
        group = group.sort_values('Frame')
        frames = group['Frame'].values  # Extracting the time and position arrays needed for displacement calculation
        x = group['X'].values     
        y = group['Y'].values

        n_points = len(frames)
        if n_points < 2:
            continue  

        max_possible_lag = n_points - 1 if max_lag is None else min(max_lag, n_points - 1)

        for lag in range(1, max_possible_lag + 1):          # This loop produces all possible displacements for a lag within the track.
            dx = x[lag:] - x[:-lag]          # computes all displacements between x-coordinates of a track separated by lag frames
            dy = y[lag:] - y[:-lag]          # computes all displacements between y-coordinates of a track separated by lag frames
            squared_disp = dx**2 + dy**2     # Computes squared displacement for each displacement
            displacements_by_lag[lag].extend(squared_disp)   # here all squared displacements for this lag from this track are added into the global dictionary

    msd_result = {lag: np.mean(disps) for lag, disps in displacements_by_lag.items()}

    # Converting to DataFrame with 'lag time' and 'MSD' columns
    msd_df = pd.DataFrame(list(msd_result.items()), columns=['lag time', 'MSD'])
    msd_df['lag time'] = msd_df['lag time'] * t_gap
    return msd_df.sort_values('lag time').reset_index(drop=True)




In [3]:
t_gap = float(input("What is the interval between each frame in seconds? "))

What is the interval between each frame in seconds? 5


# Data 1 MSD calculation:

In [4]:
data_1 = pd.read_csv('spots_17_02_24_L4440_4957_LD4um.csv', skiprows=[2,3],header=1)   # Reading the spot statistics file generated by TrackMate plugin

In [5]:
max_Total_frames=np.max(data_1['Frame'].unique())        # Finds the maximum number of frames in the given embryo's data
original_frame = list(range(1, max_Total_frames + 1))

# Asks for the time zero (furrow initiation time for this embryo)
time_zero = int(input("What is the time zero (furrow initiation frame)? "))

# "furrow initiation frame for 4957 embryo=41"

What is the time zero (Anaphase onset)? (frame number) 41


In [6]:

# Creating a sync_time list by shifting and converting to seconds
sync_time = [(frame - time_zero) * t_gap for frame in original_frame]


end_frame_1 = int(input("What is the end frame no. before which you want to calculate MSD? "))
start_frame_1 = time_zero + int(8)    #start frame is ~40s after furrow initiation.

print(f"Start frame (for MSD): {start_frame_1}")
print(f"End frame (for MSD): {end_frame_1}")

# The frame where ingression ends for this embryo is 64

What is the end frame no. before which you want to calculate MSD? 64
Start frame (for MSD): 49
End frame (for MSD): 64


In [7]:
MSD_DATA_1=compute_msd_true_average(data_1,t_gap,start_frame_1,end_frame_1)  # Using the defined function above to find MSD data for this embryo
print((MSD_DATA_1))


    lag time        MSD
0        5.0   0.591722
1       10.0   2.140659
2       15.0   4.366441
3       20.0   7.133518
4       25.0  10.536761
5       30.0  15.370886
6       35.0  20.644839
7       40.0  27.040004
8       45.0  34.626195
9       50.0  44.011736
10      55.0  57.081948
11      60.0  70.889120
12      65.0  85.061591


# Data_2 MSD calculation

In [8]:
data_2 = pd.read_csv('spots_17_02_24_L4440_4970_LD4um.csv', skiprows=[2,3],header=1)

max_Total_frames=np.max(data_2['Frame'].unique())
original_frame = list(range(1, max_Total_frames + 1))

# Asks for the time zero (furrow initiation time for this embryo)
time_zero = int(input("What is the time zero (furrow initiation frame)? "))


# Creates a sync_time list by shifting and converting to seconds
sync_time = [(frame - time_zero) * t_gap for frame in original_frame]

end_frame_2 = int(input("What is the end frame no. before which you want to calculate MSD? "))
start_frame_2 = time_zero + int(8)    # start frame is ~40s after furrow initiation.

print(f"Start frame (for MSD): {start_frame_2}")
print(f"End frame (for MSD): {end_frame_2}")



# "furrow initiation frame for embryo 4970=11"
# The frame where ingression ends for this embryo=30


What is the time zero (Anaphase onset)? (frame number) 11
What is the end frame no. before which you want to calculate MSD? 30
Start frame (for MSD): 19
End frame (for MSD): 30


In [9]:
MSD_DATA_2=compute_msd_true_average(data_2,t_gap,start_frame_2,end_frame_2)  #Using the function defined above to find MSD data


# DATA 3

In [11]:
data_3 = pd.read_csv('spots_17_02_24_L4440_4971_LD4um.csv', skiprows=[2,3],header=1)

max_Total_frames=np.max(data_3['Frame'].unique())
original_frame = list(range(1, max_Total_frames + 1))

time_zero = int(input("What is the time zero (furrow initiation frame)? "))


# Creates a sync_time list by shifting and converting to seconds
sync_time = [(frame - time_zero) * t_gap for frame in original_frame]


end_frame_3 = int(input("What is the end frame no. before which you want to calculate MSD? "))
start_frame_3 = time_zero + int(8)    #so start frame is 40s after furrow initiation.

print(f"Start frame (for MSD): {start_frame_3}")
print(f"End frame (for MSD): {end_frame_3}")



# "furrow initiation frame for 4971 embryo=49"
# The frame where ingression ends for this embryo= 75


What is the time zero (Anaphase onset)? (frame number) 49
What is the end frame no. before which you want to calculate MSD? 75
Start frame (for MSD): 57
End frame (for MSD): 75


In [12]:
MSD_DATA_3=compute_msd_true_average(data_3,t_gap,start_frame_3,end_frame_3)


# DATA 4

In [14]:
data_4 = pd.read_csv('spots_17_02_24_L4440_4985_LD4um.csv', skiprows=[2,3],header=1)

max_Total_frames=np.max(data_4['Frame'].unique())
original_frame = list(range(1, max_Total_frames + 1))

time_zero = int(input("What is the time zero (furrow initiation frame)? "))


# Creates a sync_time list by shifting and converting to seconds
sync_time = [(frame - time_zero) * t_gap for frame in original_frame]


end_frame_4 = int(input("What is the end frame no. before which you want to calculate MSD? "))
start_frame_4 = time_zero + int(8)    #start frame is 40s after furrow initiation.

print(f"Start frame (for MSD): {start_frame_4}")
print(f"End frame (for MSD): {end_frame_4}")



# "furrow initiation frame for 4985 ~ -2"
# The frame where ingression ends for this embryo= 23


What is the time zero (Anaphase onset)? (frame number) -2
What is the end frame no. before which you want to calculate MSD? 23
Start frame (for MSD): 6
End frame (for MSD): 23


In [15]:
MSD_DATA_4=compute_msd_true_average(data_4,t_gap,start_frame_4,end_frame_4)
# print(MSD_DATA_4)

# DATA 5

In [16]:
data_5 = pd.read_csv('spots_17_02_24_L4440_4992_LD4um.csv', skiprows=[2,3],header=1)

max_Total_frames=np.max(data_5['Frame'].unique())
original_frame = list(range(1, max_Total_frames + 1))

time_zero = int(input("What is the time zero (furrow initiation frame)? "))


# Creates a sync_time list by shifting and converting to seconds
sync_time = [(frame - time_zero) * t_gap for frame in original_frame]


end_frame_5 = int(input("What is the end frame no. before which you want to calculate MSD? "))
start_frame_5 = time_zero + int(8)    #start frame is ~40s after furrow initiation.

print(f"Start frame (for MSD): {start_frame_5}")
print(f"End frame (for MSD): {end_frame_5}")



# "furrow initiation frame for embryo 4992 =34"
# The frame where ingression ends for this embryo= 54


What is the time zero (Anaphase onset)? (frame number) 34
What is the end frame no. before which you want to calculate MSD? 54
Start frame (for MSD): 42
End frame (for MSD): 54


In [17]:
MSD_DATA_5=compute_msd_true_average(data_5,t_gap,start_frame_5,end_frame_5)
print(MSD_DATA_5)

    lag time        MSD
0        5.0   0.756761
1       10.0   2.258842
2       15.0   4.566375
3       20.0   7.266228
4       25.0  10.741155
5       30.0  15.892451
6       35.0  23.790399
7       40.0  31.969047
8       45.0  39.726397
9       50.0  47.741547
10      55.0  55.933516
11      60.0  63.197243


# DATA 6

In [18]:
data_6 = pd.read_csv('spots_17_02_24_L4440_4994_LD4um.csv', skiprows=[2,3],header=1)

max_Total_frames=np.max(data_6['Frame'].unique())
original_frame = list(range(1, max_Total_frames + 1))

time_zero = int(input("What is the time zero (furrow initiation frame)? "))


# Creates a sync_time list by shifting and converting to seconds
sync_time = [(frame - time_zero) * t_gap for frame in original_frame]


end_frame_6 = int(input("What is the end frame no. before which you want to calculate MSD? "))
start_frame_6 = time_zero + int(8)    #so start frame is 40s after furrow initiation.

print(f"Start frame (for MSD): {start_frame_6}")
print(f"End frame (for MSD): {end_frame_6}")



# "furrow initiation frame for embryo 4994 = 22"
# The frame where ingression ends for this embryo= 45


What is the time zero (Anaphase onset)? (frame number) 22
What is the end frame no. before which you want to calculate MSD? 45
Start frame (for MSD): 30
End frame (for MSD): 45


In [19]:
MSD_DATA_6=compute_msd_true_average(data_6,t_gap,start_frame_6,end_frame_6)
print(MSD_DATA_6)

    lag time         MSD
0        5.0    0.764700
1       10.0    2.584102
2       15.0    5.274627
3       20.0    8.925563
4       25.0   15.190608
5       30.0   22.600911
6       35.0   32.779162
7       40.0   44.998511
8       45.0   61.094384
9       50.0   83.792252
10      55.0   95.092014
11      60.0  102.441761
12      65.0  109.025147
13      70.0  113.408943
14      75.0  114.657541


# DATA 7

In [20]:
data_7 = pd.read_csv('spots_17_02_24_L4440_4995_LD4um.csv', skiprows=[2,3],header=1)

max_Total_frames=np.max(data_7['Frame'].unique())
original_frame = list(range(1, max_Total_frames + 1))

time_zero = int(input("What is the time zero (furrow initiation frame)? "))


# Creates a sync_time list by shifting and converting to seconds
sync_time = [(frame - time_zero) * t_gap for frame in original_frame]


end_frame_7 = int(input("What is the end frame no. before which you want to calculate MSD? "))
start_frame_7 = time_zero + int(8)    #start frame is 40s after furrow initiation.

print(f"Start frame (for MSD): {start_frame_7}")
print(f"End frame (for MSD): {end_frame_7}")



# "furrow initiation frame for embryo 4995=12"
# The frame where ingression ends for this embryo= 32


What is the time zero (Anaphase onset)? (frame number) 12
What is the end frame no. before which you want to calculate MSD? 32
Start frame (for MSD): 20
End frame (for MSD): 32


In [21]:
MSD_DATA_7=compute_msd_true_average(data_7,t_gap,start_frame_7,end_frame_7)
print(MSD_DATA_7)

    lag time        MSD
0        5.0   0.927650
1       10.0   2.892592
2       15.0   5.845876
3       20.0  10.671327
4       25.0  17.324869
5       30.0  24.686877
6       35.0  30.510879
7       40.0  37.267264
8       45.0  42.840707
9       50.0  48.787445
10      55.0  50.657268
11      60.0  18.623107


# DATA 8

In [22]:
data_8 = pd.read_csv('spots_17_02_24_L4440_4997_LD4um.csv', skiprows=[2,3],header=1)

max_Total_frames=np.max(data_8['Frame'].unique())
original_frame = list(range(1, max_Total_frames + 1))

time_zero = int(input("What is the time zero (furrow initiation frame)? "))


# Creates a sync_time list by shifting and converting to seconds
sync_time = [(frame - time_zero) * t_gap for frame in original_frame]


end_frame_8 = int(input("What is the end frame no. before which you want to calculate MSD? "))
start_frame_8 = time_zero + int(8)    #start frame is 40s after furrow initiation.

print(f"Start frame (for MSD): {start_frame_8}")
print(f"End frame (for MSD): {end_frame_8}")



# "furrow initiation frame for embryo 4997=30"
# The frame where ingression ends for this embryo= 52


What is the time zero (Anaphase onset)? (frame number) 30
What is the end frame no. before which you want to calculate MSD? 52
Start frame (for MSD): 38
End frame (for MSD): 52


In [23]:
MSD_DATA_8=compute_msd_true_average(data_8,t_gap,start_frame_8,end_frame_8)
print(MSD_DATA_8)

   lag time        MSD
0       5.0   0.599443
1      10.0   2.050942
2      15.0   4.287055
3      20.0   7.747187
4      25.0  12.890928
5      30.0  18.570925
6      35.0  31.178587


# DATA 9

In [24]:
data_9 = pd.read_csv('spots_17_02_24_L4440_5001_LD4um.csv', skiprows=[2,3],header=1)

max_Total_frames=np.max(data_9['Frame'].unique())
original_frame = list(range(1, max_Total_frames + 1))

time_zero = int(input("What is the time zero (furrow initiation frame)? "))


# Creates a sync_time list by shifting and converting to seconds
sync_time = [(frame - time_zero) * t_gap for frame in original_frame]


end_frame_9 = int(input("What is the end frame no. before which you want to calculate MSD? "))
start_frame_9 = time_zero + int(8)    #start frame is 40s after furrow initiation.

print(f"Start frame (for MSD): {start_frame_9}")
print(f"End frame (for MSD): {end_frame_9}")



# "furrow initiation frame for embryo 5001=14"
# The frame where ingression ends for this embryo= 39


What is the time zero (Anaphase onset)? (frame number) 14
What is the end frame no. before which you want to calculate MSD? 39
Start frame (for MSD): 22
End frame (for MSD): 39


In [25]:
MSD_DATA_9=compute_msd_true_average(data_9,t_gap,start_frame_9,end_frame_9)
print(MSD_DATA_9)

    lag time        MSD
0        5.0   0.673333
1       10.0   1.234977
2       15.0   2.021434
3       20.0   2.910371
4       25.0   3.706490
5       30.0   4.159421
6       35.0   5.445653
7       40.0   7.274895
8       45.0   8.555306
9       50.0  10.356301
10      55.0  12.710969
11      60.0  16.592204
12      65.0  22.836602
13      70.0  27.148084
14      75.0  34.703428


# DATA 10

In [26]:
data_10 = pd.read_csv('spots_17_02_24_L4440_5003_LD4um.csv', skiprows=[2,3],header=1)

max_Total_frames=np.max(data_10['Frame'].unique())
original_frame = list(range(1, max_Total_frames + 1))

time_zero = int(input("What is the time zero (furrow initiation frame)? "))


# Creates a sync_time list by shifting and converting to seconds
sync_time = [(frame - time_zero) * t_gap for frame in original_frame]


end_frame_10 = int(input("What is the end frame no. before which you want to calculate MSD? "))
start_frame_10 = time_zero + int(8)    #start frame is 40s after furrow initiation.

print(f"Start frame (for MSD): {start_frame_10}")
print(f"End frame (for MSD): {end_frame_10}")



# "furrow initiation frame for embryo 5003=10"
# The frame where ingression ends for this embryo= 28


What is the time zero (Anaphase onset)? (frame number) 10
What is the end frame no. before which you want to calculate MSD? 28
Start frame (for MSD): 18
End frame (for MSD): 28


In [27]:
MSD_DATA_10=compute_msd_true_average(data_10,t_gap,start_frame_10,end_frame_10)
print(MSD_DATA_10)

   lag time        MSD
0       5.0   0.616541
1      10.0   2.086855
2      15.0   4.010868
3      20.0   6.871788
4      25.0  11.040657
5      30.0  16.512939
6      35.0  21.435875
7      40.0  30.404584
8      45.0  39.870899
9      50.0  47.412234


# DATA 11

In [28]:
data_11 = pd.read_csv('spots_L4440_4575_frame_0_280_LD4um.csv', skiprows=[2,3],header=1)

max_Total_frames=np.max(data_11['Frame'].unique())
original_frame = list(range(1, max_Total_frames + 1))

time_zero = int(input("What is the time zero (furrow initiation frame)? "))


# Creates a sync_time list by shifting and converting to seconds
sync_time = [(frame - time_zero) * t_gap for frame in original_frame]


end_frame_11 = int(input("What is the end frame no. before which you want to calculate MSD? "))
start_frame_11 = time_zero + int(8)    #so start frame is 40s after furrow initiation.

print(f"Start frame (for MSD): {start_frame_11}")
print(f"End frame (for MSD): {end_frame_11}")



# "furrow initiation frame for embryo 4575=31"
# The frame where ingression ends for this embryo= 53


What is the time zero (Anaphase onset)? (frame number) 31
What is the end frame no. before which you want to calculate MSD? 53
Start frame (for MSD): 39
End frame (for MSD): 53


In [29]:
MSD_DATA_11=compute_msd_true_average(data_11,t_gap,start_frame_11,end_frame_11)
print(MSD_DATA_11)

    lag time         MSD
0        5.0    1.220040
1       10.0    4.466272
2       15.0   10.343364
3       20.0   19.179096
4       25.0   30.501381
5       30.0   47.508084
6       35.0   74.506625
7       40.0   94.508297
8       45.0  113.294741
9       50.0  127.985848
10      55.0  139.293633


# DATA 12

In [34]:
data_12 = pd.read_csv('spots_L4440_4576_frame_0_89_LD4um.csv', skiprows=[2,3],header=1)

max_Total_frames=np.max(data_12['Frame'].unique())
original_frame = list(range(1, max_Total_frames + 1))

time_zero = int(input("What is the time zero (furrow initiation frame)? "))


# Creates a sync_time list by shifting and converting to seconds
sync_time = [(frame - time_zero) * t_gap for frame in original_frame]


end_frame_12 = int(input("What is the end frame no. before which you want to calculate MSD? "))
start_frame_12 = time_zero + int(8)    # start frame is 40s after furrow initiation.

print(f"Start frame (for MSD): {start_frame_12}")
print(f"End frame (for MSD): {end_frame_12}")



# "furrow initiation frame for embryo 4576=40" not very clear
# The frame where ingression ends for this embryo= 60


What is the time zero (Anaphase onset)? (frame number) 40
What is the end frame no. before which you want to calculate MSD? 60
Start frame (for MSD): 48
End frame (for MSD): 60


In [35]:
MSD_DATA_12=compute_msd_true_average(data_12,t_gap,start_frame_12,end_frame_12)
print(MSD_DATA_12)

   lag time       MSD
0       5.0  0.537403
1      10.0  1.473940


# DATA 13

In [36]:
data_13 = pd.read_csv('spots_L4440_4587_frame_0_120_LD4um.csv', skiprows=[2,3],header=1)

max_Total_frames=np.max(data_13['Frame'].unique())
original_frame = list(range(1, max_Total_frames + 1))

time_zero = int(input("What is the time zero (furrow initiation frame)? "))


# Creates a sync_time list by shifting and converting to seconds
sync_time = [(frame - time_zero) * t_gap for frame in original_frame]


end_frame_13 = int(input("What is the end frame no. before which you want to calculate MSD? "))
start_frame_13 = time_zero + int(8)    #start frame is 40s after furrow initiation.

print(f"Start frame (for MSD): {start_frame_13}")
print(f"End frame (for MSD): {end_frame_13}")



# "furrow initiation frame for embryo 4587=42"
# The frame where ingression ends for this embryo= 68


What is the time zero (Anaphase onset)? (frame number) 42
What is the end frame no. before which you want to calculate MSD? 68
Start frame (for MSD): 50
End frame (for MSD): 68


In [37]:
MSD_DATA_13=compute_msd_true_average(data_13,t_gap,start_frame_13,end_frame_13)
print(MSD_DATA_13)

    lag time         MSD
0        5.0    0.722424
1       10.0    2.316757
2       15.0    4.978859
3       20.0    8.571640
4       25.0   13.377182
5       30.0   19.154206
6       35.0   25.926032
7       40.0   34.838694
8       45.0   46.637523
9       50.0   59.801447
10      55.0   76.918071
11      60.0   93.439083
12      65.0  108.985930
13      70.0  124.792523
14      75.0  139.388960
15      80.0  174.282765
16      85.0  190.097499


# DATA 14

In [38]:
data_14 = pd.read_csv('spots_L4440_4589_frame_0_91_LD4um.csv', skiprows=[2,3],header=1)

max_Total_frames=np.max(data_14['Frame'].unique())
original_frame = list(range(1, max_Total_frames + 1))

time_zero = int(input("What is the time zero (furrow initiation frame)? "))


# Creates a sync_time list by shifting and converting to seconds
sync_time = [(frame - time_zero) * t_gap for frame in original_frame]


end_frame_14 = int(input("What is the end frame no. before which you want to calculate MSD? "))
start_frame_14 = time_zero + int(8)    # start frame is 40s after furrow initiation.

print(f"Start frame (for MSD): {start_frame_14}")
print(f"End frame (for MSD): {end_frame_14}")



# "furrow initiation frame for embryo 4589=22"
# The frame where ingression ends for this embryo= 43


What is the time zero (Anaphase onset)? (frame number) 22
What is the end frame no. before which you want to calculate MSD? 43
Start frame (for MSD): 30
End frame (for MSD): 43


In [39]:
MSD_DATA_14=compute_msd_true_average(data_14,t_gap,start_frame_14,end_frame_14)
print(MSD_DATA_14)

   lag time        MSD
0       5.0   0.868502
1      10.0   2.919981
2      15.0   5.444972
3      20.0   9.786501
4      25.0  17.020777
5      30.0  27.330561
6      35.0  29.636038
7      40.0  33.287759


# DATA 15

In [40]:
data_15 = pd.read_csv('spots_L4440_4590_frame_0_80_LD4um.csv', skiprows=[2,3],header=1)

max_Total_frames=np.max(data_15['Frame'].unique())
original_frame = list(range(1, max_Total_frames + 1))

time_zero = int(input("What is the time zero (furrow initiation frame)? "))


# Creates a sync_time list by shifting and converting to seconds
sync_time = [(frame - time_zero) * t_gap for frame in original_frame]


end_frame_15 = int(input("What is the end frame no. before which you want to calculate MSD? "))
start_frame_15 = time_zero + int(8)    #start frame is 40s after furrow initiation.

print(f"Start frame (for MSD): {start_frame_15}")
print(f"End frame (for MSD): {end_frame_15}")



# "furrow initiation frame for embryo 4590=6"
# The frame where ingression ends for this embryo= 31


What is the time zero (Anaphase onset)? (frame number) 6
What is the end frame no. before which you want to calculate MSD? 31
Start frame (for MSD): 14
End frame (for MSD): 31


In [41]:
MSD_DATA_15=compute_msd_true_average(data_15,t_gap,start_frame_15,end_frame_15)
print(MSD_DATA_15)

    lag time        MSD
0        5.0   0.490691
1       10.0   1.312148
2       15.0   2.327631
3       20.0   3.700833
4       25.0   5.325351
5       30.0   6.894596
6       35.0   8.042826
7       40.0   8.972002
8       45.0   9.733600
9       50.0  10.642448
10      55.0  12.449877
11      60.0  14.689834
12      65.0  16.786636
13      70.0  17.649061
14      75.0  19.687221
15      80.0  20.585945
16      85.0  20.841889


# combining all data frames

In [42]:
combined_MSD_dataframe = pd.concat([MSD_DATA_1, MSD_DATA_2, MSD_DATA_3, MSD_DATA_4, MSD_DATA_5, MSD_DATA_6, MSD_DATA_7, MSD_DATA_8, MSD_DATA_9, MSD_DATA_10, MSD_DATA_11, MSD_DATA_12, MSD_DATA_13, MSD_DATA_14, MSD_DATA_15],axis=1) #Pandas will align them by the index ("lag time")# Missing lag times in any DataFrame will show up as NaN (not available) in those columns



In [68]:
print(combined_MSD_dataframe)

    lag time       MSD  lag time       MSD  lag time        MSD  lag time  \
0        5.0  0.484004       5.0  0.718833       5.0   0.656592       5.0   
1       10.0  0.972845      10.0  1.576305      10.0   1.457892      10.0   
2       15.0  1.159901      15.0  2.425957      15.0   2.504565      15.0   
3       20.0  1.850311      20.0  3.125757      20.0   3.527098      20.0   
4       25.0  2.673358      25.0  3.432774      25.0   4.972555      25.0   
5       30.0  4.001338      30.0  2.893987      30.0   6.761618      30.0   
6       35.0  5.762372      35.0  1.062299      35.0   8.854009      35.0   
7       40.0  7.139429      40.0  1.741806      40.0  10.753677      40.0   
8        NaN       NaN       NaN       NaN      45.0  13.191341      45.0   
9        NaN       NaN       NaN       NaN      50.0  15.488553      50.0   
10       NaN       NaN       NaN       NaN      55.0  17.632341      55.0   
11       NaN       NaN       NaN       NaN      60.0  17.433123      60.0   

In [43]:
combined_MSD_dataframe.to_csv("control_combined_MSD_file_for_different_end_time_during_ingression_after_40s.csv")

# print("Combined MSD DataFrame saved as 'control_combined_MSD_file_for_different_end_time_during_ingression_after_40s.csv'.")

Combined MSD DataFrame saved as 'L4440_combined_MSD_file_for_different_end_time_during_ingression_after_40s.csv'.


In [ ]:
msd_data_list = [
    MSD_DATA_1, MSD_DATA_2, MSD_DATA_3, MSD_DATA_4, MSD_DATA_5,
    MSD_DATA_6, MSD_DATA_7, MSD_DATA_8, MSD_DATA_9, MSD_DATA_10,MSD_DATA_10, MSD_DATA_11, MSD_DATA_12, MSD_DATA_13, MSD_DATA_14, MSD_DATA_15
]

# Applying natural log to each DataFrame and store in a new list
log_msd_data_list = [np.log(df) for df in msd_data_list]

# Concatenating all log-transformed MSD DataFrames
combined_log_MSD_dataframe = pd.concat(log_msd_data_list, axis=1)

combined_log_MSD_dataframe =combined_log_MSD_dataframe.rename(columns={'MSD': 'Ln(MSD)'})
combined_log_MSD_dataframe =combined_log_MSD_dataframe.rename(columns={'lag time': 'Ln(lag time)'})


In [ ]:
# combined_log_MSD_dataframe.to_csv("control_RZB135_medial_combined_ln_MSD_40s_to_different_end_time_during_ingression.csv")
